In [3]:
# Adapted from: https://gist.github.com/SamratSahoo/cef04a39a4033f7bec0299a10701eb95

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import cv2

In [2]:
# This function will let us standardize out image inputs
def processImage(image, scale=1): 
    # open the image
    image = cv2.imread(image) 
    # make it grayscale
    image = cv2.cvtColor(src=image, code=cv2.COLOR_BGR2GRAY) 
    # resize it
    image= cv2.resize(image, (0,0), fx = scale, fy = scale)
    return image

In [4]:
# This function will do our convolution
def convolve2D(image, kernel, padding=0, strides=1):
    # Cross Correlation
    kernel = np.flipud(np.fliplr(kernel))

    # Gather Shapes of Kernel + Image + Padding
    xKernShape = kernel.shape[0]
    yKernShape = kernel.shape[1]
    xImgShape = image.shape[0]
    yImgShape = image.shape[1]

    # Shape of Output Convolution
    xOutput = int(((xImgShape - xKernShape + 2 * padding) / strides) + 1)
    yOutput = int(((yImgShape - yKernShape + 2 * padding) / strides) + 1)
    output = np.zeros((xOutput, yOutput))

    # Apply Equal Padding to All Sides
    if padding != 0:
        imagePadded = np.zeros((image.shape[0] + padding*2, image.shape[1] + padding*2))
        imagePadded[int(padding):int(-1 * padding), int(padding):int(-1 * padding)] = image
        print(imagePadded)
    else:
        imagePadded = image

    # Iterate through image
    for y in range(image.shape[1]):
        # Exit Convolution
        if y > image.shape[1] - yKernShape:
            break
        # Only Convolve if y has gone down by the specified Strides
        if y % strides == 0:
            for x in range(image.shape[0]):
                # Go to next row once kernel is out of bounds
                if x > image.shape[0] - xKernShape:
                    break
                try:
                    # Only Convolve if x has moved by the specified Strides
                    if x % strides == 0:
                        output[x, y] = (kernel * imagePadded[x: x + xKernShape, y: y + yKernShape]).sum()
                except:
                    break

    return output

In [5]:
# Make the kernel

# Open the image and make it grayscale
k_path = './img/logo.png'
k_data = processImage(k_path, 0.5)

# Make it pure black (0) and white (1)
logo_kernel = np.zeros(np.shape(k_data))
for i in range(np.shape(k_data)[0]):
    for j in range(np.shape(k_data)[1]):
        if k_data[i][j] < 180:
            logo_kernel[i][j] = 1
            
# Let's see what that looks like
fig, ax = plt.subplots(nrows=1, ncols=2)
ax[0].imshow(k_data)
ax[1].imshow(logo_kernel)
plt.show()